In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.misc as misc
from skimage.transform import resize as imresize
import numpy as np
from scipy.io import loadmat as load_from_matlab

In [2]:
def imread(path):
    return imresize(plt.imread(path).astype(np.float32)/255, (244, 244), mode='reflect')

def imbatch1(img):
    return np.expand_dims(img, axis=0)


matvgg19 = load_from_matlab('/home/diogo/data/deep_studies/style_transfer/imagenet-vgg-verydeep-19.mat')
matclasses = matvgg19['classes'][0, 0] 
matlayers = matvgg19['layers'][0]

def read_classes(matclasses):
    idxs, names = matclasses
    idxs = idxs[0]
    names = names[0]
    fname = [n[0].split(',')[0] for n in names]
    
    return fname, names, idxs, 

name, _, _, = read_classes(matclasses)




In [3]:
layer = matlayers[0]
layer[0, 0]['name']

array(['conv1_1'], dtype='<U7')

In [5]:
tf.reset_default_graph()

STRIDE1 = (1, 1, 1, 1)
PIXEL_SCALE = float(255)
PIXEL_OFFSET = np.array([103.939, 116.779, 123.68])


def conv(inputs, matlayer):
    """
    Build an convolutional layer from matlab weitghs on data.
    """

    # Raise an error before is too late ;)
    W, B = matlayer['weights'][0]
    It, Ih, Iw, Ic = inputs.shape
    Ww, Wh, Wi, Wo = W.shape
    assert Wi == Ic
    assert B.shape == (1, Wo)

    # Matlab to tensorflow
    W = W.transpose((1, 0, 2, 3))
    B = B.reshape(-1)

    tfW = tf.constant(W)
    tfB = tf.constant(B)
    return tf.nn.conv2d(inputs, tfW, STRIDE1, 'SAME') + tfB


def relu(inputs, matlayer):
    # Raise an error before is too late.
    assert matlayer['type'][0] == 'relu'

    return tf.nn.relu(inputs)


def maxpool(inputs, matlayer):
    # Yep, another error check.
    assert matlayer['type'][0] == 'pool'

    return tf.layers.max_pooling2d(inputs, 2, 2, padding='VALID')


def flatten(inputs, matlayer):
    # Better sooner than late.
    W, B = matlayer['weights'][0]
    It, Ih, Iw, Ic = inputs.shape
    Ww, Wh, Wi, Wo = W.shape
    assert (Wh, Ww, Wi) == (Ih, Iw, Ic)
    assert B.shape == (1, Wo)
    
    return tf.layers.flatten(inputs)


def dense(inputs, matlayer):
    # Yo know what they say about precautoins.
    W, B = matlayer['weights'][0]
    Wh, Ww, Wi, Wo = W.shape
    It, Il = inputs.shape
    Wri, Wro = W.reshape([-1, Wo]).shape
    assert (Wh * Ww * Wi) == Wri
    assert Wro == Wo    
    assert B.shape == (1, Wo)
    assert Wri == Il
    
    tfW = tf.constant(W.reshape([-1, Wo]))
    tfB = tf.constant(B)
    return tf.matmul(inputs, tfW) + tfB


def softmax(inputs, matlayer):
    # I know, i'm getting old, so what?
    assert matlayer['type'][0] == 'softmax'
    
    return tf.nn.softmax(inputs)


from collections import OrderedDict

layers = OrderedDict()
tf_input_image = tf.placeholder(shape=[1, 244, 244, 3], dtype=tf.float32)
x = PIXEL_SCALE * tf_input_image - PIXEL_OFFSET

layers['input'] = tf_input_image
layers['processed_input'] = x


mat_layers_map = {
    'conv': conv,
    'relu': relu,
    'pool': maxpool,
    'dense': dense,
    'softmax': softmax,
}

usetop = True
verbose = False

def layerTypeName(matlayer):
    ltype, lname = matlayer['type'][0], matlayer['name'][0]
    
    # Optimizes fully connected layers as tensorflow dense layer
    if ltype == 'conv' and lname.startswith('fc'):
        ltype = 'dense'
    
    return ltype, lname

for matlayer in matlayers:
    matlayer = matlayer[0, 0]
    ltype, lname = layerTypeName(matlayer)    
    if lname == 'fc6':
        if usetop:
            if verbose:
                print('Creating layer{} of type {}'.format('flatten', 'reshape'))

            x = flatten(x, matlayer)
            layers['flatten'] = x
        else:
            break

    if verbose:
        print('Reading layer{} of type{}'.format(lname, ltype))
            
    x = mat_layers_map[ltype](x, matlayer)
    layers[lname] = x


sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [7]:
img = imread('/home/diogo/data/deep_studies/style_transfer/tusker.jpeg')
imgbt = imbatch1(img)
with sess.as_default():
    npclasses = sess.run(x, feed_dict={tf_input_image:imgbt})
    idxs = np.argsort(npclasses)
    print(list(reversed(list(zip(np.asarray(name)[idxs[0, -5:]], npclasses[0, idxs[0, -5:]])))))


[('tusker', 0.14157006), ('Indian elephant', 0.11259125), ('African elephant', 0.096443094), ('triceratops', 0.08624849), ('water buffalo', 0.07027102)]


In [33]:
XX = npclasses#p.concatenate([npclasses, npclasses[:, ::-1]])
print(XX.shape)

idxs = np.argsort(XX)[:, -5:]
#XX[idxs]

#np.partition(XX, 2)

#print(XX)
print(XX[0, idxs[0, ::-1]])
#print(XX[1, idxs[1, ::-1]])

(1, 1000)
[0.14157006 0.11259125 0.09644309 0.08624849 0.07027102]


In [ ]:
mat_layers_map['relu']

#matlayers = matvgg19['layers'][0]
#matlayers[0]['weights'][0, 0][0].shape

In [ ]:
from src.autoencodernet import StyleAutoEncoder

In [ ]:
tf.reset_default_graph()


tfx = tf.layers.Input(shape=(256, 256, 3))
tfxrec = StyleAutoEncoder(tfx)


loss = tf.nn.l2_loss(tfx - tfxrec.output)#/(256*256*3)
train = tf.train.AdamOptimizer(0.1).minimize(loss)


sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
with sess.as_default():
    nptfx = sess.run(tfxrec.output, feed_dict={tfx: 255 * np.random.random([1, 256, 256, 3])})

In [ ]:
nptfx.shape

In [ ]:
plt.imshow(nptfx[0])

In [ ]:
print(nptfx.min(), nptfx.max())

In [ ]:
img = imread('/home/diogo/data/deep_studies/style_transfer/night.jpeg')

In [ ]:
print(img.min(), img.max())
plt.imshow(img)

In [ ]:
ll = []

In [ ]:
import os
files = os.listdir('/home/diogo/data/deep_studies/style_transfer/fast-style-transfer/data/train2014')

def plot_img_cmp(X, Xt, nimages=10):
    imgidxs = np.random.choice(X.shape[0], nimages)

    fig, axeslist = plt.subplots(ncols=nimages, nrows=2, sharex=True, sharey=True)
    fig.set_size_inches(16, 8)
    #fig.subplots_adjust(left=0, bottom=0, right=1, top=0.1, wspace=0, hspace=0)
    
    for i in range(nimages):
        axeslist[0, i].imshow(X[imgidxs[i]])
        axeslist[0, i].set_axis_off()
        
        axeslist[1, i].imshow(Xt[imgidxs[i]])
        axeslist[1, i].set_axis_off()

    fig.subplots_adjust(wspace=0, hspace=-0.7)

In [ ]:
%matplotlib inline
from IPython.display import display, clear_output
base = '/home/diogo/data/deep_studies/style_transfer/fast-style-transfer/data/train2014/'

food = {tfx: 255*imbatch1(img)}
with sess.as_default():
    for i in range(50):
        npimages = np.asarray([255 * imread(base + x) for x in np.random.choice(files, 5)], dtype=np.float32)
        food = {tfx: npimages}
        l, _ = sess.run([loss, train], feed_dict=food)
        ll.append(l)
        if i % 5 == 0 and i > 0:
            imgt = sess.run(tfxrec.output, feed_dict=food)
            imgt = imgt
            #imgt = np.squeeze(imgt, 0)
            #clear_output()
            #plt.ioff()
            #_ll = ll[:min(len(ll), 200)]
            #plt.plot(np.arange(len(_ll)), _ll)
            plt.plot(np.arange(len(ll)), ll)
            plt.show()
            
            #plot_img_random(Xt)
            plot_img_cmp(npimages/255, imgt/255, 3)
            #plt.imshow(imgt/255)
            plt.show()
            clear_output(wait=True)
        


In [ ]:
print(imgt.min(), imgt.max())

In [ ]:
len(files)

In [ ]:
X =  np.asarray([255 * imread('/home/diogo/data/deep_studies/style_transfer/fast-style-transfer/data/train2014/' + x) for x in np.random.choice(files, 10)], dtype=np.float32)

fig, axeslist = plt.subplots(ncols=5, nrows=1, sharey=True)
for i in range(5):
    fig.set_size_inches(14, 3)
    axeslist.ravel()[i].imshow(X[i]/255)
    axeslist.ravel()[i].axis('off')


In [ ]:
plt.imshow(X[1]/255)

In [ ]:
X.shape

In [ ]:
def need_input(layer):
    def new_function(s, *args, **kwargs):
        original_function(*args, **kwargs)
        print("Hello, galaxy!")
    return new_function

In [ ]:
tf.reset_default_graph()

In [ ]:
weights = data['layers'][0]

In [ ]:
weights[37][0][0][0][0][0].shape

In [ ]:

tf.reset_default_graph()

STRIDE1 = (1, 1, 1, 1)
PIXEL_SCALE = float(255)
PIXEL_OFFSET = np.array([103.939, 116.779, 123.68])


def conv(data, filters, size, inputs):
    """
    Build an convolutional layer from matlab weitghs on data.
    """

    # Raise an error before is too late ;)
    W, B = data[0][0][0][0]
    It, Ih, Iw, Ic = inputs.shape
    assert W.shape == (size, size, Ic, filters)
    assert B.shape == (1, filters)

    # Matlab to tensorflow
    W = W.transpose((1, 0, 2, 3))
    B = B.reshape(-1)

    tfW = tf.constant(W)
    tfB = tf.constant(B)
    return tf.nn.conv2d(inputs, tfW, STRIDE1, 'SAME') + tfB


def relu(data, inputs):
    # Raise an error before is too late.
    assert data[0][0][0][0] == 'relu'

    return tf.nn.relu(inputs)


def maxpool(data, inputs):
    # Yep, another error check.
    assert 'pool' in data[0][0][0][0]

    return tf.layers.max_pooling2d(inputs, 2, 2, padding='VALID')


def flatten(data, inputs):
    # Better sooner than late.
    W, B = data[0][0][0][0]
    It, Ih, Iw, Ic = inputs.shape
    Ww, Wh, Wi, Wo = W.shape
    assert (Wh, Ww, Wi) == (Ih, Iw, Ic)
    assert B.shape == (1, Wo)
    
    return tf.layers.flatten(inputs)


def dense(data, inputs):
    # Yo know what they say about precautoins.
    W, B = data[0][0][0][0]
    Wh, Ww, Wi, Wo = W.shape
    It, Il = inputs.shape
    Wri, Wro = W.reshape([-1, Wo]).shape
    assert (Wh * Ww * Wi) == Wri
    assert Wro == Wo    
    assert B.shape == (1, Wo)
    assert Wri == Il
    
    tfW = tf.constant(W.reshape([-1, Wo]))
    tfB = tf.constant(B)
    return tf.matmul(inputs, tfW) + tfB


def softmax(data, inputs):
    # I know, i'm getting old, so what?
    assert 'softmax' == data[0][0][0][0]
    
    return tf.nn.softmax(inputs)



tf_input_image = tf.placeholder(shape=[1, 244, 244, 3], dtype=tf.float32)
x = PIXEL_SCALE * tf_input_image - PIXEL_OFFSET


#for layerspec in weights:
#    layerspec = layerspec[0, 0]
#    ltype = layerspec['type'][0]
#    lname = layerspec['name'][0]
#    
#    if ltype == 'conv':
#        print(ltype, lname, layerspec['weights'][0, 0].shape, layerspec['weights'][0, 1].shape, layerspec['pad'], layerspec['stride'])
#    elif ltype == 'pool':
#        print(ltype, lname, layerspec['method'], layerspec['pad'], layerspec['stride'])
#    elif ltype == 'relu':
#        print(ltype, lname)
#    else:
#        print(layerspec.dtype, layerspec['type'], layerspec['name']
#
#    if lname == 'pool5':
#        break


#x = 255 * tf.reverse(tf_input_image, axis=[-1]) - offset

#x = tf_input_image

x = conv(weights[0], 64, 3, x)
x = relu(weights[1], x)
x = conv(weights[2], 64, 3, x)
x = relu(weights[3], x)
x = maxpool(weights[4], x)

x = conv(weights[5], 128, 3, x)
x = relu(weights[6], x)
x = conv(weights[7], 128, 3, x)
x = relu(weights[8], x)
x = maxpool(weights[9], x)

x = conv(weights[10], 256, 3, x)
x = relu(weights[11], x)
x = conv(weights[12], 256, 3, x)
x = relu(weights[13], x)
x = conv(weights[14], 256, 3, x)
x = relu(weights[15], x)
x = conv(weights[16], 256, 3, x)
x = relu(weights[17], x)
x = maxpool(weights[18], x)

x = conv(weights[19], 512, 3, x)
x = relu(weights[20], x)
x = conv(weights[21], 512, 3, x)
x = relu(weights[22], x)
x = conv(weights[23], 512, 3, x)
x = relu(weights[24], x)
x = conv(weights[25], 512, 3, x)
x = relu(weights[26], x)
x = maxpool(weights[27], x)

x = conv(weights[28], 512, 3, x)
x = relu(weights[29], x)
x = conv(weights[30], 512, 3, x)
x = relu(weights[31], x)
x = conv(weights[32], 512, 3, x)
x = relu(weights[33], x)
x = conv(weights[34], 512, 3, x)
x = relu(weights[35], x)
x = maxpool(weights[36], x)

x = flatten(weights[37], x)
x = dense(weights[37], x)
x = relu(weights[38], x)
x = dense(weights[39], x)
x = relu(weights[40], x)
x = dense(weights[41], x)
tf_output_classes = softmax(weights[42], x)

assert x.shape == (1, 1000)

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
img = imread('/home/diogo/data/deep_studies/style_transfer/tusker.jpeg')
#plt.imshow(img)
imgbt = imbatch1(img)
#print(imgbt.min(), imgbt.max(), imgbt.shape)

#imgbt = imgbt[:, :, :, (2, 1, 0)] - np.asarray([123.68 , 116.779, 103.939])# np.asarray([103.939, 116.779, 123.68])/255

tt = (0, 1, 2)

#imgbt = 255*imgbt[:, :, :, tt] - np.asarray([103.939, 116.779, 123.68])

#print(imgbt.min(), imgbt.max(), imgbt.shape)


#imgbt = imgbt[(0, 3, 2, 1) ] - np.asarray([103.939, 116.779, 123.68])/255

#print(type(imgbt), type(tf_output_classes), type(tf_input_image))

with sess.as_default():
    npclasses = sess.run(tf_output_classes, feed_dict={tf_input_image:imgbt})
    idxs = np.argsort(npclasses)
    print(list(reversed(list(zip(np.asarray(name)[idxs[0, -5:]], npclasses[0, idxs[0, -5:]])))))

In [ ]:
W = np.zeros((7, 7, 512, 4096))

In [ ]:
W, B = weights[37][0][0][0][0]

In [ ]:
W.shape

In [ ]:
data['normalization']

In [ ]:
def read_classes(data):
    idxs, names = data['classes'][0, 0]
    idxs = idxs[0]
    names = names[0]
    fname = [n[0].split(',')[0] for n in names]
    
    return fname, names, idxs, 

name, _, _, = read_classes(data)


In [ ]:
name[100]

In [ ]:
names[0]

In [ ]:
#RGB -> BGR
#(0, 1, 2) -> (2, 1, 0) 

np.asarray([103.939, 116.779, 123.68])/255

In [ ]:
weights[37][0, 0].dtype

In [ ]:

for layerspec in weights:
    layerspec = layerspec[0, 0]
    ltype = layerspec['type'][0]
    lname = layerspec['name'][0]
    if ltype == 'conv':
        print(ltype, lname, layerspec['weights'][0, 0].shape, layerspec['weights'][0, 1].shape, layerspec['pad'], layerspec['stride'])
    elif ltype == 'pool':
        print(ltype, lname, layerspec['method'], layerspec['pad'], layerspec['stride'])
    elif ltype == 'relu':
        print(ltype, lname)
    else:
        print(layerspec.dtype, layerspec['type'], layerspec['name'])



In [ ]:
weights[0][0, 0]['pad']

In [ ]:
weights[0][0, 0]['stride']

In [ ]:
tf.reset_default_graph()

PIXEL_SCALE = float(255)
PIXEL_OFFSET = np.array([103.939, 116.779, 123.68])

def conv(inputs, matlayer):
    """
    Build an convolutional layer from matlab weitghs on data.
    """

    # Raise an error before is too late ;)
    W, B = matlayer['weights'][0]
    stride = np.ones([1,4], dtype=np.int)
    stride[[1,2]] = matlayer['stride'][0]
    pad = matlayer['pad'][0]

    Ww, Wh, Wi, Wo = W.shape
    It, Ih, Iw, Ic = inputs.shape
    assert Wi == Ic
    assert B.shape == (1, Wo)

    # Matlab to tensorflow
    W = W.transpose((1, 0, 2, 3))
    B = B.reshape(-1)

    tfW = tf.constant(W)
    tfB = tf.constant(B)
    return tf.nn.conv2d(inputs, tfW, stride, 'SAME') + tfB



def conv(data, filters, size, inputs):
    W, B = data[0][0][0][0]
    T, H, Ci, C = inputs.shape
    
    assert W.shape == (size, size, C, filters)
    assert B.shape == (1, filters)
    assert C == W.shape[2]
    
    print('OK!')
    
    W = W.transpose((1, 0, 2, 3))
    B = B.reshape(-1)


    tfW = tf.constant(W)
    tfB = tf.constant(B)

    _conv = tf.nn.conv2d(inputs, tfW, STRIDE1, 'SAME') + tfB
    print(inputs.shape, '->', _conv.shape)
    return _conv

def relu(data, inputs):
    assert data[0][0][0][0] == 'relu'
    print('OK!')
    return tf.nn.relu(inputs)

def maxpool(data, inputs):
    assert 'pool' in data[0][0][0][0]
    
    _l = tf.layers.max_pooling2d(inputs, 2, 2, padding='VALID')
    
    print(inputs.shape, '->', _l.shape)
    return _l


def flatten(data, inputs):
    W, B = data[0][0][0][0]
    Tx, Hx, Wx, Cx = inputs.shape
    WH, WW, Wi, Wo = W.shape

    print(inputs.shape, W.shape, B.shape)
    
    assert (WH, WH, Wi) == (Hx, Wx, Cx)
    print('OK!')
    return tf.layers.flatten(inputs)


def dense(data, x):
    W, B = data[0][0][0][0]
    WH, WH, Wi, Wo = W.shape
    W = W.reshape([-1, Wo])
    
    I, O = W.shape

    assert B.shape == (1, Wo)
    
    T, L = x.shape
    
    assert L == I
    print('OK!')
    return tf.matmul(x, W) + B
    
def softmax(data, x):
    assert 'softmax' == data[0][0][0][0]
    print('OK!')
    return tf.nn.softmax(x)



In [ ]:
vgg19specs['classes'][0, 0][2]